In [1]:
import scanpy as sc
import torch

In [2]:
import anndata
import numpy as np
import copy
import matplotlib.pyplot as plt

from scipy.stats import spearmanr
from scvi.external import GIMVI

In [3]:
def preprocess(vis2):
    vis2.var_names_make_unique
    vis2.X=np.log(((vis2.X/np.sum(vis2.X))*1000000) + 1)
    return(vis2)

In [4]:
rna=sc.read_h5ad("/data2/duren_lab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/rnap21.h5ad")
atac=sc.read_h5ad("/data2/duren_lab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/atacp21.h5ad")

In [5]:
sprna=sc.read_h5ad("/data2/duren_lab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/sprnap21.h5ad")

In [6]:
sprna=preprocess(sprna)
atac=preprocess(atac)
rna=preprocess(rna)

In [7]:
sc_multi=anndata.concat([rna, atac], axis=1)

In [8]:
sc_multi.X

array([[0.        , 0.        , 0.69420511, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.40617052, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.25427395, ..., 0.40597698, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [9]:
GIMVI.setup_anndata(sprna)
GIMVI.setup_anndata(sc_multi)

/data2/duren_lab/idebnat/conda_env/scvi_env/lib/python3.9/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)
/data2/duren_lab/idebnat/conda_env/scvi_env/lib/python3.9/site-packages/scvi/data/fields/_base_field.py:64: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [10]:
# create our model
model = GIMVI(sc_multi, sprna)

# train for 200 epochs
model.train(200)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/data2/duren_lab/idebnat/conda_env/scvi_env/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/data2/duren_lab/idebnat/conda_env/scvi_env/lib/python3.9/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 1/200:   0%|          | 0/200 [00:00<?, ?it/s]

/data2/duren_lab/idebnat/conda_env/scvi_env/lib/python3.9/site-packages/scvi/external/gimvi/_module.py:359: UserWarning: The value argument must be within the support of the distribution
  -ZeroInflatedNegativeBinomial(mu=px_rate, theta=px_r, zi_logits=px_dropout)


Epoch 200/200: 100%|██████████| 200/200 [19:29<00:00,  5.78s/it, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [19:29<00:00,  5.85s/it, v_num=1]


In [11]:
_, spatac_imputed=model.get_imputed_values()

In [12]:
spatac=sc.read_h5ad("/data2/duren_lab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/spatacp21.h5ad")

In [46]:
# spatac=preprocess(spatac)

In [13]:
spatac_imputed=spatac_imputed[:,:atac.var.shape[0]]

In [14]:
anndata.AnnData(spatac_imputed, obs=sprna.obs, var=atac.var).write_h5ad("gimvi_spatac.h5ad")